<a href="https://colab.research.google.com/github/blkeyd/CVPR/blob/main/Final/Paper/AlexNet_FireDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Image transformation

In [3]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

Dataset Split

In [4]:
import os
import shutil
import random

source_dir = "/content/drive/MyDrive/fire_dataset"
target_dir = "/content/drive/MyDrive/fire_dataset_split"

classes = ["fire", "nofire"]
split_ratio = [0.7, 0.15, 0.15]  # train, val, test

# Create folders
for split in ["train", "val", "test"]:
    for cls in classes:
        os.makedirs(os.path.join(target_dir, split, cls), exist_ok=True)

# Split files
for cls in classes:
    cls_path = os.path.join(source_dir, cls)
    images = os.listdir(cls_path)
    random.shuffle(images)

    total = len(images)
    train_end = int(total * split_ratio[0])
    val_end = train_end + int(total * split_ratio[1])

    train_imgs = images[:train_end]
    val_imgs = images[train_end:val_end]
    test_imgs = images[val_end:]

    for img in train_imgs:
        shutil.copy(os.path.join(cls_path, img),
                    os.path.join(target_dir, "train", cls, img))

    for img in val_imgs:
        shutil.copy(os.path.join(cls_path, img),
                    os.path.join(target_dir, "val", cls, img))

    for img in test_imgs:
        shutil.copy(os.path.join(cls_path, img),
                    os.path.join(target_dir, "test", cls, img))

print("Dataset successfully split into train / val / test")

Dataset successfully split into train / val / test


Load Dataset

In [5]:
data_dir = "/content/drive/MyDrive/fire_dataset_split"

train_data = datasets.ImageFolder(data_dir + "/train", transform=transform)
val_data   = datasets.ImageFolder(data_dir + "/val", transform=transform)
test_data  = datasets.ImageFolder(data_dir + "/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False)

class_names = train_data.classes
print(class_names)   # ['fire', 'nofire']

['fire', 'nofire']


Load AlexNet

In [6]:
model = models.alexnet(pretrained=True)

# Modify last layer for 2 classes
model.classifier[6] = nn.Linear(4096, 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 130MB/s]


Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Training Loop

In [10]:
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss/len(train_loader):.4f} "
              f"Train Acc: {train_acc:.2f}% "
              f"Val Acc: {val_acc:.2f}%")


Run Training

In [12]:
train_model(model, train_loader, val_loader, epochs=10)

Epoch [1/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [2/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [3/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [4/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [5/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [6/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [7/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [8/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [9/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%
Epoch [10/10] Train Loss: 0.0000 Train Acc: 100.00% Val Acc: 99.33%


Testing Accuracy

In [13]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [14]:
test_model(model, test_loader)

Test Accuracy: 96.71%


In [15]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:\n", cm)
    print("\nClassification Report:\n",
          classification_report(all_labels, all_preds, target_names=class_names))

evaluate_model(model, test_loader)


Confusion Matrix:
 [[110   4]
 [  1  37]]

Classification Report:
               precision    recall  f1-score   support

        fire       0.99      0.96      0.98       114
      nofire       0.90      0.97      0.94        38

    accuracy                           0.97       152
   macro avg       0.95      0.97      0.96       152
weighted avg       0.97      0.97      0.97       152

